In [1]:
import numpy as np
#from scipy.signal import butter, filtfilt
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline
#import seaborn as sns
#sns.set_context("notebook", font_scale=1.3,
#                rc={'lines.linewidth': 1.5, 'lines.markersize': 14, 'axes.titlesize': 'x-large'})
#matplotlib.rc('legend', numpoints=1, fontsize=14)
#import glob
import sys, os
sys.path.insert(1, r'./Functions')
# IPython widgets:
from IPython.display import display
import ipywidgets
from ipywidgets import FloatProgress, interactive
import seaborn as sns
from scipy.signal import detrend
import pickle

In [2]:
#!pip install version_information
%load_ext version_information
%version_information numpy, scipy, pandas, matplotlib, seaborn

Software versions
Python 3.8.8 64bit [MSC v.1916 64 bit (AMD64)]
IPython 7.22.0
OS Windows 10 10.0.22621 SP0
numpy 1.20.1
scipy 1.6.2
pandas 1.2.4
matplotlib 3.3.4
seaborn 0.11.1
Fri May 19 12:21:40 2023 Hora oficial do Brasil

In [3]:
# GitHub URL:
#path2 = 'C:\Users\ferna\Downloads\IC- dados\Dados'
#local directory:
path2 = r'C:\\Users\\ferna\\Downloads\\IC- dados\\Dados'
#path2 = '/home/rnwatanabe/copdata'
fname = os.path.join(path2, 'PDSinfo.txt')
PDSinfo = pd.read_csv(fname, sep='\t', header=0, index_col=None, engine='c', encoding='utf-8')
print(fname)
print("Information of %s subjects loaded (%s rows, %s columns)."
      %(len(pd.unique(PDSinfo.Subject)), PDSinfo.shape[0], PDSinfo.shape[1]))

C:\\Users\\ferna\\Downloads\\IC- dados\\Dados\PDSinfo.txt
Information of 49 subjects loaded (588 rows, 29 columns).


In [4]:
a=PDSinfo[PDSinfo['Subject']==44].index.tolist()
e=PDSinfo[PDSinfo['Subject']==44].index.tolist()[2]
ei = PDSinfo[PDSinfo['Subject']==44]
PDSinfo.memory_usage()

Index                 128
Trial                4704
Subject              4704
Vision               4704
Surface              4704
Rep                  4704
Age                  4704
AgeGroup             4704
Gender               4704
Height               4704
Mass                 4704
BMI                  4704
FootLen              4704
DominantLeg          4704
Nationality          4704
SkinColor            4704
Ystudy               4704
Footwear             4704
Illness              4704
Illness2             4704
Nmedication          4704
Medication           4704
Ortho-Prosthesis     4704
Ortho-Prosthesis2    4704
Disability           4704
Disability2          4704
Falls12m             4704
PhysicalActivity     4704
Sequence             4704
Date                 4704
dtype: int64

In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
fs = 100 # Hz
import pickle 
#file = open('important', 'wb')

COPX = np.zeros((49*12, 60*fs-30))
COPZ = np.zeros((49*12, 60*fs-30))

COPXreal = np.zeros((49*12, 60*fs-30))
COPYreal = np.zeros((49*12, 60*fs-30))
COPZreal = np.zeros((49*12, 60*fs-30))

d1x = np.zeros((49*12, 60*fs-30))
d2x = np.zeros((49*12, 60*fs-30))

d1z = np.zeros((49*12, 60*fs-30))
d2z = np.zeros((49*12, 60*fs-30))

matrizangx = np.zeros((49*12, 60*fs-30))
matrizangz = np.zeros((49*12, 60*fs-30))

alpha = np.zeros((49*12, 60*fs-30))


COGX = np.empty((49*12, 60*fs-30))
print(COGX.shape)
COGY = np.empty((49*12, 60*fs-30))
COGZ = np.empty((49*12, 60*fs-30))

d = np.zeros((49*12, 60*fs-30))


angulox = np.zeros((49*12, 60*fs-30))
anguloz= np.zeros((49*12, 60*fs-30))

matrizangx = np.zeros((49*12, 60*fs-30))
matrizangz = np.zeros((49*12, 60*fs-30))



subject = np.zeros((0,1))
j=0


trial = 0
for s in range (1,50):
    
    #a = PDSinfo[PDSinfo['Subject']==s].index.tolist()[2]
    
    ei = PDSinfo[PDSinfo['Subject']==s].index.tolist()
    
    
    for i in range(0,len(ei)):
        a = ei[i]
        fname_grf = os.path.join(path2, PDSinfo.Trial[a] + 'grf' + '.txt') 
        fname_mkr = os.path.join(path2, PDSinfo.Trial[a] + 'mkr' + '.txt')
        grf = pd.read_csv(fname_grf, delimiter='\t', header=0, engine='c')
        mkr = pd.read_csv(fname_mkr, delimiter='\t', header=0, engine='c')
        
        
        COGX[trial,:]=(mkr['COG_X'].values[30:])
        COGY[trial,:]=(mkr['COG_Y'].values[30:])
        COGZ[trial,:]=(mkr['COG_Z'].values[30:])
        
        
        mass=PDSinfo['Mass']
        if((np.isnan(d)).any) != 'True':
            d[trial,:] = np.sqrt(COGX[trial,:]**2+COGY[trial,:]**2+COGZ[trial,:]**2)
            angulox[trial,:] = np.arcsin(COGX[trial,:]/d[trial,:])
            anguloz[trial,:] = np.arcsin(COGZ[trial,:]/d[trial,:])
            sinx = np.sin(angulox[trial,:])
            sinz = np.sin(anguloz[trial,:])
            cosx = np.cos(angulox[trial,:])
            cosz = np.cos(anguloz[trial,:])
            tanx = np.tan(angulox[trial,:])
            tanz = np.tan(anguloz[trial,:])
            matrizangx[trial,:] = angulox[trial,:]
            matrizangz[trial,:] = anguloz[trial,:]


            Jb=(4/3)*(mass.values[s])*((d[trial,:])**2)
            t=grf['Time']
            dt=t[1]-t[0]


            d1x[trial,:] = np.gradient(matrizangx[trial,:],dt)
            d1z[trial,:] = np.gradient(matrizangz[trial,:],dt)

            d2x[trial,:] = np.gradient(d1x[trial,:],dt)
            d2z[trial,:] = np.gradient(d1z[trial,:],dt)

            COPX[trial,:] = d[trial,:]*sinx + Jb/(mass.values[s]*9.8)*(sinx*d1x[trial,:]**2-cosx*d2x[trial,:]) 
            COPZ[trial,:] = d[trial,:]*sinz + Jb/(mass.values[s]*9.8)*(sinz*d1z[trial,:]**2-cosz*d2z[trial,:])




            COPXreal[trial,:] = grf['COPNET_X'].values[30:]
            COPZreal[trial,:] = grf['COPNET_Z'].values[30:]

            subject = np.vstack((subject, [s]))

            trial = trial + 1

            #ASISXnovo = np.vstack((ASISXnovo,(ASISX[-1,:])[np.logical_not(np.isnan(ASISX[-1,:]))]))
            #ASISYnovo = np.vstack((ASISYnovo,(ASISY[-1,:])[np.logical_not(np.isnan(ASISY[-1,:]))]))
            #ASISZnovo = np.vstack((ASISZnovo,(ASISZ[-1,:])[np.logical_not(np.isnan(ASISZ[-1,:]))]))


            #if (ASISXnovo.size)!=0:
       



file = open('filename', 'wb')

         
pickle.dump((COPXreal,COPZreal,COPX,COPZ,d1x,d1z,d2x,d2z,matrizangx,matrizangz,COGX,COGY,COGZ,angulox,anguloz,d,alpha,subject,ei,sinx,sinz,cosx,cosz,tanx,tanz), file)


file.close()




(588, 5970)


In [6]:
file = open('filename','rb')
COPXreal = pickle.load(file)
COPZreal = pickle.load(file)
COPX = pickle.load(file)
COPZ = pickle.load(file)
d1x = pickle.load(file)
d1z = pickle.load(file)
d2x = pickle.load(file)
d2z = pickle.load(file)
matrizangx = pickle.load(file)
matrizangz =pickle.load(file)
COGX = pickle.load(file)
COGY = pickle.load(file)
COGZ = pickle.load(file)
angulox = pickle.load(file)
anguloz = pickle.load(file)
d = pickle.load(file)
alpha = pickle.load(file)
subject = pickle.load(file)
ei = pickle.load(file)
sinx = pickle.load(file)
sinz = pickle.load(file)
cosx = pickle.load(file)
cosz = pickle.load(file)
tanx = pickle.load(file)
tanz =pickle.load(file)
file.close()

EOFError: Ran out of input

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
fs = 100 # Hz
import pickle 
#file = open('important', 'wb')

COPX = np.zeros((49*12, 60*fs-30))
COPZ = np.zeros((49*12, 60*fs-30))

COPXreal = np.zeros((49*12, 60*fs-30))
COPYreal = np.zeros((49*12, 60*fs-30))
COPZreal = np.zeros((49*12, 60*fs-30))

d1x = np.zeros((49*12, 60*fs-30))
d2x = np.zeros((49*12, 60*fs-30))

d1z = np.zeros((49*12, 60*fs-30))
d2z = np.zeros((49*12, 60*fs-30))

matrizangx = np.zeros((49*12, 60*fs-30))
matrizangz = np.zeros((49*12, 60*fs-30))

alpha = np.zeros((49*12, 60*fs-30))


ASISX = np.empty((49*12, 60*fs-30))
ASISY = np.empty((49*12, 60*fs-30))
ASISZ = np.empty((49*12, 60*fs-30))

mass = np.empty((49*12))

d = np.zeros((49*12, 60*fs-30))

angulox = np.zeros((49*12, 60*fs-30))
anguloz= np.zeros((49*12, 60*fs-30))

matrizangx = np.zeros((49*12, 60*fs-30))
matrizangz = np.zeros((49*12, 60*fs-30))



subject = np.zeros((0,1))
j=0


trial = 0
for s in range (1,50):
    
    #a = PDSinfo[PDSinfo['Subject']==s].index.tolist()[2]
    
    ei = PDSinfo[PDSinfo['Subject']==s].index.tolist()
    
    
    for i in range(0,len(ei)):
        a = ei[i]
        fname_grf = os.path.join(path2, PDSinfo.Trial[a] + 'grf' + '.txt') 
        fname_mkr = os.path.join(path2, PDSinfo.Trial[a] + 'mkr' + '.txt')
        grf = pd.read_csv(fname_grf, delimiter='\t', header=0, engine='c')
        mkr = pd.read_csv(fname_mkr, delimiter='\t', header=0, engine='c')
        
        #mkr.dropna(0,'any',inplace=True)
        #grf.dropna(0,'any',inplace=True)
        
        ASISX[trial,:]=(mkr['R.ASIS_X'].values[30:])
        ASISY[trial,:]=(mkr['R.ASIS_Y'].values[30:])
        ASISZ[trial,:]=(mkr['R.ASIS_Z'].values[30:])
        
        
        
        mass[trial,] = (PDSinfo['Mass']).values[s]
        
        d[trial,:] = np.sqrt(ASISX[trial,:]**2+ASISY[trial,:]**2+ASISZ[trial,:]**2)
        if (any(np.isnan(d[trial,:]))) != 'True':
            angulox[trial,:] = np.arcsin(ASISX[trial,:]/d[trial,:])
            anguloz[trial,:] = np.arcsin(ASISZ[trial,:]/d[trial,:])
            sinx = np.sin(angulox[trial,:])
            sinz = np.sin(anguloz[trial,:])
            cosx = np.cos(angulox[trial,:])
            cosz = np.cos(anguloz[trial,:])
            tanx = np.tan(angulox[trial,:])
            tanz = np.tan(anguloz[trial,:])
            matrizangx[trial,:] = angulox[trial,:]
            matrizangz[trial,:] = anguloz[trial,:]
            


            Jb=(4/3)*(mass[trial,])*((d[trial,:])**2)
            t=grf['Time']
            dt=t[1]-t[0]


            d1x[trial,:] = np.gradient(matrizangx[trial,:],dt)
            d1z[trial,:] = np.gradient(matrizangz[trial,:],dt)

            d2x[trial,:] = np.gradient(d1x[trial,:],dt)
            d2z[trial,:] = np.gradient(d1z[trial,:],dt)

            COPX[trial,:] = d[trial,:]*sinx + Jb/(mass[trial,]*9.8)*(sinx*d1x[trial,:]**2-cosx*d2x[trial,:]) 
            COPZ[trial,:] = d[trial,:]*sinz + Jb/(mass[trial,]*9.8)*(sinz*d1z[trial,:]**2-cosz*d2z[trial,:])




            COPXreal[trial,:] = grf['COPNET_X'].values[30:]
            COPZreal[trial,:] = grf['COPNET_Z'].values[30:]

            
            subject = np.vstack((subject, [s]))

            trial = trial + 1
        
        #ASISXnovo = np.vstack((ASISXnovo,(ASISX[-1,:])[np.logical_not(np.isnan(ASISX[-1,:]))]))
        #ASISYnovo = np.vstack((ASISYnovo,(ASISY[-1,:])[np.logical_not(np.isnan(ASISY[-1,:]))]))
        #ASISZnovo = np.vstack((ASISZnovo,(ASISZ[-1,:])[np.logical_not(np.isnan(ASISZ[-1,:]))]))
        
        
        #if (ASISXnovo.size)!=0:



filename = open('file_name', 'wb')

pickle.dump((COPXreal,COPZreal,COPX,COPZ,d1x,d1z,d2x,d2z,matrizangx,matrizangz,ASISX,ASISY,ASISZ,angulox,anguloz,d,alpha,subject,ei,sinx,sinz,cosx,cosz,tanx,tanz), filename)


filename.close()



In [ ]:
filename = open('file_name','rb')

COPXreal = pickle.load(filename)
COPZreal = pickle.load(filename)
COPX = pickle.load(filename)
COPZ = pickle.load(filename)
d1x = pickle.load(filename)
d1z = pickle.load(filename)
d2x = pickle.load(filename)
d2z = pickle.load(filename)
matrizangx = pickle.load(filename)
matrizangz =pickle.load(filename)
ASISX = pickle.load(filename)
ASISY = pickle.load(filename)
ASISZ = pickle.load(filename)
angulox = pickle.load(filename)
anguloz = pickle.load(filename)
d = pickle.load(filename)
alpha = pickle.load(filename)
subject = pickle.load(filename)
ei = pickle.load(filename)
sinx = pickle.load(filename)
sinz = pickle.load(filename)
cosx = pickle.load(filename)
cosz = pickle.load(filename)
tanx = pickle.load(filename)
tanz =pickle.load(filename)


filename.close()

In [ ]:


testex=np.where(np.isnan(COGX))
testey=np.where(np.isnan(COGY))
testez=np.where(np.isnan(COGZ))

cogx=np.delete(COGX,[testex[0]],0)
cogy=np.delete(COGY,[testey[0]],0)
cogz=np.delete(COGZ,[testez[0]],0)

subjectc = np.delete(subject,[testex[0]],0)

testex = np.where(np.isnan(ASISX))
testey = np.where(np.isnan(ASISY))
testez = np.where(np.isnan(ASISZ))

asisx=np.delete(ASISX,[testex[0]],0)
asisy=np.delete(ASISY,[testey[0]],0)
asisz=np.delete(ASISZ,[testez[0]],0)


subjecta = np.delete(subject,[testex[0]],0)



testex=np.where(np.isnan(COPX))
testez=np.where(np.isnan(COPZ))

copx=np.delete(COPX,[testex[0]],0)
copz=np.delete(COPZ,[testez[0]],0)
subjectp = np.delete(subject,[testex[0]],0)

print(copx.shape)
print(subjectp.shape)

print(asisx.shape)
print(subjecta.shape)

In [ ]:
import time
import timeit

inicio = timeit.default_timer()

indices = np.arange(1,50).reshape(-1,1)
np.random.shuffle(indices)
train = indices[:40]
test = indices[40:]


#masstrain = mass.values[train]
masstrain = mass[np.in1d(subject,train),]
print(train.shape)

masstest = mass[np.in1d(subject,test),]

ASISXtrain = asisx[np.in1d(subjecta,train),]
COGXtrain = cogx[np.in1d(subjectc,train),]
COGXtest = cogx[np.in1d(subjectc,test),]
ASISXtest = asisx[np.in1d(subjecta,test),]
COPXtrain = copx[np.in1d(subjectp,train),]
COPXtest = copx[np.in1d(subjectp,test),]

ASISYtrain = asisy[np.in1d(subjecta,train),]
COGYtrain = cogy[np.in1d(subjectc,train),]
COGYtest = cogy[np.in1d(subjectc,test),]
ASISYtest = asisy[np.in1d(subjecta,test),]

ASISZtrain = asisz[np.in1d(subjecta,train),]
ASISZtest = asisz[np.in1d(subjecta,test),]
COPZtrain = copz[np.in1d(subjectp,train),]
COPZtest = copz[np.in1d(subjectp,test),]
COGZtrain = cogz[np.in1d(subjectc,train),]
COGZtest = cogz[np.in1d(subjectc,test),]

fim = timeit.default_
print(asisx.shape)
print(cogx.shape)
print(COPX.shape)

In [ ]:
def newdist(paramt,x,y,z):
    from sklearn.model_selection import train_test_split
    from scipy.optimize import minimize, rosen, rosen_der
    from sklearn.metrics import mean_squared_error
    dest = np.zeros((60*fs-30))
    d = np.zeros((60*fs-30))
    for i in range(0, len(x)):

        delta,mi = paramt
        
        d[i] = np.mean(np.sqrt(x[i]**2+y[i]**2+z[i]**2))
        
        dest[i] = delta * d[i] + mi
    
      
    return dest


In [ ]:
#for i in range(0,len(COGXtrain)):
        
        #d= np.sqrt(COGXtrain[i]**2+COGYtrain[i]**2+COGZtrain[i]**2)
        
def meansquarerrordist(paramt,d,ASISXtrain,ASISYtrain,ASISZtrain):
    d = np.zeros((60*fs-30))
    from sklearn.metrics import mean_squared_error
    ytrue = np.zeros((60*fs-30))
    ypredicted = np.zeros((60*fs-30))
    for i in range(0,len(ASISXtrain)):
        
        d[i] = np.mean(np.sqrt(COGXtrain[i]**2+COGYtrain[i]**2+COGZtrain[i]**2))
        
        ytrue[i] = d[i]
        
        ypredicted = newdist(paramt,ASISXtrain,ASISYtrain,ASISZtrain)
        
        
    return mean_squared_error(ytrue, ypredicted)


In [ ]:
from sklearn.model_selection import train_test_split
#from scipy import optmize
from scipy.optimize import minimize, rosen, rosen_der
first_guess = [1,1]
res = minimize(meansquarerrordist, first_guess,(d,COGXtrain,COGYtrain,COGZtrain))
coeficientes = res.x
delta=coeficientes[0]
mi=coeficientes[1]
print(delta)
print(mi)

In [ ]:
treino = newdist(coeficientes,COGXtrain,COGYtrain,COGZtrain)
teste = newdist(coeficientes,COGXtest,COGYtest,COGZtest)

Regressão linear distância

In [ ]:
def centerofpressure(paramt, M,x,y,z):
    from sklearn.model_selection import train_test_split
    from scipy.optimize import minimize, rosen, rosen_der
    from sklearn.metrics import mean_squared_error
    COPXES = np.empty((49*12, 60*fs-30))
    COPXESorig =np.empty((49*12, 60*fs-30))
    dest =np.empty((49*12,60*fs-30))
    trial = 0 
    sennovox=np.empty((49*12,60*fs-30))
    cosnovox =np.empty((49*12,60*fs-30))
    sennovoz=np.empty((49*12,60*fs-30))
    cosnovoz =np.empty((49*12,60*fs-30))
    
    for i in range(0,len(x)):
        
        A,alpha,beta, gama= paramt
        
        d[trial,:] = np.mean(np.sqrt(x[trial,:]**2+y[trial,:]**2+z[trial,:]**2))
        
        dest[trial,:] = delta * d[trial,:] + mi
        
        angulox[trial,:] = np.arcsin(np.tanh(x[trial,:]/dest[trial,:]))
        anguloz[trial,:] = np.arcsin(np.tanh(z[trial,:]/dest[trial,:]))
        sinx=np.sin(angulox[trial,:])
        sinz=np.sin(anguloz[trial,:])
        cosx=np.cos(angulox[trial,:])
        cosz=np.cos(anguloz[trial,:])
        tanx=np.tan(angulox[trial,:])
        tanz=np.tan(anguloz[trial,:])
            
        matrizangx[trial,:] = angulox[trial,:]
        matrizangz[trial,:] = anguloz[trial,:]







        sennovox[trial,:] = (sinx)*A + ((1-A**2)**1/2)*(cosx)
            

        cosnovox[trial,:] = (cosx)*A - ((1-A**2)**1/2)*(sinx)
            
        sennovoz[trial,:] = (sinz)*A + ((1-A**2)**1/2)*(cosz)
           

        cosnovoz[trial,:] = (cosz)*A - ((1-A**2)**1/2)*(sinz)

        



        Jb=(4/3)*(M[trial])*((dest[trial,:])**2)
            
        t=grf['Time']
        dt=t[1]-t[0]
            
        d1x[trial,:]=np.gradient(matrizangx[trial,:],dt)
        d1z[trial,:]=np.gradient(matrizangz[trial,:],dt)
            
            
        d2x[trial,:]=np.gradient(d1x[trial,:],dt)
        d2z[trial,:]=np.gradient(d1z[trial,:],dt)
            
        COPX[trial,:] = dest[trial,:]*sennovox[trial,:] + Jb/(M[trial]*9.8)*(sennovox[trial,:]*d1x[trial,:]**2-cosnovox[trial,:]*d2x[trial,:])
        COPZ[trial,:] = dest[trial,:]*sennovoz[trial,:] + Jb/(M[trial]*9.8)*(sennovoz[trial,:]*d1z[trial,:]**2-cosnovoz[trial,:]*d2z[trial,:])

        COPXES[trial,:]= alpha*(dest[trial,:]*np.tanh(sennovox[trial,:]) + ((Jb/(M[trial]*9.8))*np.tanh(sennovox[trial,:])*d1x[trial,:]**2)) + d2x[trial,:]*(-alpha*np.tanh(cosnovox[trial,:])*Jb/(M[trial]*9.8)+beta) + gama
        COPXESorig[trial,:]= COPX[trial,:]
        
        
        trial = trial +1
        
    
    return COPXES



In [ ]:
def meansquarerror(paramt,COPXtrain,masstrain,ASISXtrain,ASISYtrain,ASISZtrain):
    
    from sklearn.metrics import mean_squared_error
    
    ytrue = np.empty((49*12, 60*fs-30))
    A,alpha,beta, gama = paramt
    for i in range(0,len(ASISXtrain)):
        
        ytrue[i] = COPXtrain[i]
    
        ypredicted = centerofpressure(paramt,masstrain,ASISXtrain,ASISYtrain,ASISZtrain)
    
        
    
    
    return mean_squared_error(ytrue, ypredicted)
        


    

In [ ]:
from sklearn.model_selection import train_test_split
#from scipy import optmize
from scipy.optimize import minimize, rosen, rosen_der
first_guess = [1,1,1,1]
res = minimize(meansquarerror, first_guess,(COPXtrain,masstrain,ASISXtrain,ASISYtrain,ASISZtrain))
coeficientes = res.x
A=coeficientes[0]
alpha=coeficientes[1]
beta=coeficientes[2]
gama=coeficientes[3]

Test

In [ ]:

treinando=centerofpressure(coeficientes, masstrain,ASISXtrain,ASISYtrain,ASISZtrain) 

testando=centerofpressure(coeficientes, masstest,ASISXtest,ASISYtest,ASISZtest)


In [ ]:
time = np.arange(0,6000)/100
for i in range(0,len(ASISXtest)): b
    plt.figure()
    plt.plot(time[:5970],detrend(testando[i,:]),color='purple',label='COP predicted')
    plt.plot(time[:5970],detrend(COPXreal[i]),color='blue', label='COP real')
    plt.title('COP real X COP calculated for test {:.1f}'.format(i))
    plt.xlabel('Time')
    plt.legend()
    plt.show()

In [ ]:
sennovo = sinx*A + ((1-A**2)**1/2)*cosx
print(np.max(sennovo))
print(np.min(sennovo))

In [ ]:
cosnovo = cosx*A - ((1-A**2)**1/2)*sinx
print(np.max(cosnovo))
print(np.min(cosnovo))